In [1]:

# 🚀 STEP 2: Check if CUDA is available
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


CUDA Available: True
Device: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [6]:
!pip install unsloth
!pip install bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: Could not find a version that satisfies the requirement triton (from versions: none)
ERROR: No matching distribution found for triton


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/991.5 kB ? eta -:--:--
   --------------------- ------------------ 524.3/991.5 kB 1.4 MB/s eta 0:00:01
   ------------------------------- -------- 786.4/991.5 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 1.6 MB/s eta 0:00:00


In [4]:
pip install peft==0.15.1 accelerate -U


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from datasets import load_dataset

# Load TinyLlama model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
)

# Prepare model for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Apply LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # typical for attention
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, peft_config)

# Load your dataset
dataset = load_dataset("ahmed000000000/cybersec", split="train")

# Format the dataset
def format_prompt(example):
    prompt = f"User: {example['INSTRUCTION']}\nAI: {example['RESPONSE']}"
    return {"text": prompt}

dataset = dataset.map(format_prompt)
dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=512), batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Training arguments
training_args = TrainingArguments(
    output_dir="tinyllama-cybersec-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=1e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    lr_scheduler_type="cosine",
    warmup_steps=10,
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

# Train
trainer.train()

# Save model
model.save_pretrained("LEVI")
tokenizer.save_pretrained("LEVI")


C:\Users\alpha\anaconda3\envs\cyber\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\alpha\anaconda3\envs\cyber\lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'Cache' from 'transformers' (C:\Users\alpha\anaconda3\envs\cyber\lib\site-packages\transformers\__init__.py)